# How DyND Views Memory

## Basics of the DyND Architecture Part 1

<p style="text-align:right">
Mark Wiebe<br/>
Continuum Analytics
</p>

# Motivating the DyND Array Data Structure

To work with DyND at a low level, particularly to be able to develop in the library, one needs a good understanding of how DyND views memory. We're going to work our way from Python objects to DyND arrays, motivating its design as we go.

At the end of these slides, you will hopefully grasp why DyND is structured as it is, and be prepared to dig deeper into its code.

# Python Objects

We begin with a look at a Python object. In Python, objects are stored in memory blocks independently allocated on the heap. A single floating point value looks something like this:

<center>![Python Float](fig/python_float.svg)</center>

This doesn’t look too bad, in a dynamically typed system we need to store some metadata about the type, and we need to store the data containing the value.

# Python List

Often, however, we don’t want just one floating point value, we want a whole array of them. In Python, this is done using a list object, which looks like this:

<center>![Python List of Float](fig/python_list_of_float.svg)</center>

# Python List Problems

We can start to see some problems from this picture.

* The float type metadata is repeated seven times. No way to say “everything is float, just store that once.”
* The memory for the floats might be separated or out of order in memory. Bad for CPU cache utilization.
* There is memory being used for the pointers to all the individual floats.

We will call this the “Smalltalk road.” Program data is made of many little objects, connected by pointers. Another option is the “Fortran road,” where program data is made of arrays of data, stored contiguously.

# Smalltalk Road vs Fortran Road

There's a long but worthwhile set of lectures by Alexander Stepanov on Amazon A9's youtube site which goes into some depth about many programming topics. 


In [1]:
from IPython.display import YouTubeVideo
YouTubeVideo("Y4qdNddSWAg", start=190)

# Why Python Is Slow

Jake VanderPlas has [an excellent blog post](https://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/) which dives into more details of Python's structure, that of the CPython interpreter in particular.

The game programming industry is a great place to look for performance inspiration, as they work under constraints far more harsh than most developers. Bob Nystrom's book "Game Programming Patterns" [has a chapter about data locality](http://gameprogrammingpatterns.com/data-locality.html) which goes into nice depth on the topic.



# Dynamic Array

Let's take the Python list we looked at, factor out the redundant repetition of the float type metadata, and split all the type information away from the value storage. This gives us an abstract idea of how a dynamic array should look:

<center>![Dynamic Array](fig/abstract_array_of_float.svg)</center>

# NumPy Array

The dynamic array we made is just a small step away from the NumPy array. In NumPy, the “Array of 7” part is represented as (ndim, shape, strides), and the “Float type” part is represented as the NumPy dtype.

<center>![NumPy Array](fig/numpy_array_of_float.svg)</center>

# NumPy Strided Array

The strided array approach NumPy takes turns out to be a very good data structure for many kinds of numeric data. Allowing the strides to be set arbitrarily means both C-order (row-major) and Fortran-order (column-major) data can be represented naturally. There is good material out there going into more depth showing the consequences and benefits of it.

One good resource is [this section of open SciPy lecture notes.](https://scipy-lectures.github.io/intro/numpy/array_object.html)

<center><img src="https://scipy-lectures.github.io/_images/numpy_indexing.png" width="500"/></center>

# Weaknesses of NumPy

NumPy, as fantastic as it is, has proved to be lacking in many areas. It provides something good enough and stable enough that a huge ecosystem has grown on top of it, but there are many directions people try to stretch it that require incredible contortions or performance sacrifices to achieve.

A few of these weaknesses include

* A slow pace of evolution. NumPy development is highly constrained by backwards compatibility requirements.
* Written directly against the CPython API. There is no core library that can be shared in a larger community. (Compare with [Jupyter](http://jupyter.org/))
* Implemented in C, which tends to be very verbose and error-prone relative to its expressiveness. C++, especially with the recent C++11 and C++14 standards, has many features that help implementing this kind of library.
* Missing features that people want: Ragged arrays, more data types, labeled dimensions, and physical units metadata, to name a few.

# The DyND Array

DyND begins one step back from NumPy in the data representation narrative we have followed, generalizing the “Dynamic Array” we saw differently. Instead of focusing on the “Array of 7” part and turning it into a “general strided multi-dimensional array,” DyND treats it as one dimension of a hierarchically represented type. DyND takes the regularities in the types of stored data, bundles them together into a type, and takes the data as a separate bundle. This gets us a picture like this:

<center>![DyND Array Take 1](fig/dynd_array_of_float_take_1.svg)</center>

# DyND Types

The types DyND is using are from datashape, a type system created for the Blaze project. What datashape does is combine the shape and dtype notions we saw in the NumPy array.

The shape is generalized slightly, allowing both NumPy-style strided dimensions, called “fixed” as seen in the DyND Array diagram, as well as variable-sized dimensions, called “var.” This variable-sized dimension allows for storage of ragged data.

The dtype is also slightly generalized over NumPy, but more importantly a language is defined for writing out types in the datashape system. This is spelled out in [the datashape documentation](http://datashape.pydata.org/grammar.html).

# Slicing Arrays

Slicing is a pretty common operation to perform on numerical arrays, so we're going to use it to examine how the structure of the DyND array we defined fares.

In NumPy, slicing is enabled by the strided nature of the array object and the ability to point at the data of another array object. DyND works much the same way. Each “fixed” dimension has an associated stride, and the DyND array contains a data reference in addition to the raw data pointer.

Our diagram didn't include the stride, though, so we have to determine where it will go.

Based on what was described so far, there are two places: in the type or in the value. Let’s first imagine we put the stride in the type, and see what happens when we apply two different slices to an array.

# Putting The Stride In The Type

To take a look at what will happen, let's consider an array A with three elements, [1, 2, 3], and type “fixed[size=3, stride=8] * float64”.

<center>![The Array A](fig/dynd_array_of_float_type_contains_stride.svg)</center>

# Slice First Two Elements

Our first slice grabs first two elements, in Python it is A[:2]. This produces an array with a new type “fixed[size=2, stride=8] * float64”, and points to the same data as A.

<center>![A sliced first two elements](fig/dynd_array_of_float_type_contains_stride_slice_first_two.svg)</center>

# Slice Every Second Element

Our second slice grabs every second element, in Python it is A[::2]. This produces an array with a new type “fixed[size=2, stride=16] * float64”, and points to the same data as A.

<center>![A sliced first two elements](fig/dynd_array_of_float_type_contains_stride_slice_every_second.svg)</center>

# Problems With Stride In The Type

By slicing different ways, we’ve produced two arrays which are “arrays with two floats,” but have different types. In NumPy, this distinction is hidden away in the strides value attached to the object, and the transparent handling of the this detail is considered a feature, not a problem. It would be nicer to just be able to say we want a “fixed dimension of size 2” or just a “fixed dimension.”

DyND's usual strided arrays do not include the stride in the type. The types are spelled "2 \* float64" and "fixed \* float64".

# Putting The Stride In The Data

The other place we had available for the stride information is with the data. There are multiple ways we could do this, but let’s just consider the most obvious one, storing the stride immediately followed by the data. This means our original array looks like this:

<center>![DyND Array with Stride in the Data](fig/dynd_array_of_float_data_contains_stride.svg)</center>

# Slice First Two Elements

Our first slice, A[:2], still looks good. The only things that change in the diagram are the “fixed[3]” dimension becomes “fixed[2]”, and the third value in the array is ignored.

<center>![A sliced first two elements](fig/dynd_array_of_float_data_contains_stride_slice_first_two.svg)</center>

# Slice Every Second Element

Our second slice, however, leads to trouble. We want to view the same data, but now with a different stride. The stride must remain 8 for the original A, but needs to be 16 for the sliced A[::2], leading to the conclusion that the set of arrays viewed by this representation is not closed under slicing.

<center>![A sliced every second element](fig/dynd_array_of_float_data_contains_stride_slice_every_second.svg)</center>

Another trivial yet problematic case is slicing the last two elements, where the place for the stride would overlap with a value.

# Where To Put The Stride?

Since neither the type nor the data of the array seem like appropriate places to put the stride, we need a new location for it. Recall that in NumPy, the array object contains a dtype, a data pointer, and a few additional values including ndim, shape, and strides. We’ve put ndim and shape into the type, but we’re going to leave strides in the array object, creating some new space called “array metadata” or “arrmeta” for short.

While the exercise we just went through may feel like a triviality, it’s partly an attempt to get you thinking about how various data types get laid out in memory. In this example, just placing the stride ahead of the data leads to something absurd, but there are other cases like the “var” dimension type, which are structured pretty close to this.

# Array Metadata (Arrmeta)

Every dynd type has the opportunity to store an arbitrary but fixed amount of arrmeta in the array object. This is a place to store information that doesn't really belong in the type, but is also necessary to fully describe the data the array object is pointing at.

In the case of the fixed dimension type, we have chosen to store both the size and the stride of the dimension. (The size is redundant, but included for simple low-level access.) For other types, other arrmeta is needed. We’re going to examine what arrmeta we should choose for a struct/record type, by thinking about some operations we want to perform on data of that type.

# A DyND Struct

Let’s start with a simple 2D point type, with an X and a Y coordinate. In datashape, we can spell it as “{X: float64, Y: float64}”. If we treat it as a C/C++ struct, here’s how an array A of this type looks:

<center>![DyND Struct Take 1](fig/dynd_struct_take_1.svg)</center>

# Accessing A Struct Field

Just like with the slicing operation we had before, we have simple operations we want to support, such as field selection. If we access field A.Y, we get

<center>![A.Y](fig/dynd_struct_accessed_field_y.svg)</center>

# A More Complicated Struct

A struct with only two fields isn't enough to illustrate our next operation, of selecting multiple fields. In addition to the coordinates, our data might include differentials as well, so let's add them.

<center>![More Complicated Struct Take 1](fig/dynd_struct_take_1_complicated.svg)</center>

# Selecting Multiple Fields

Suppose we have a function which takes values of type “{X: float64, Y: float64}” as a parameter, and we want to pass the X and Y coordinates from our complicated struct to that function. A natural thing to try is to select these two fields to get a new struct.

<center>![More Complicated Struct Take 1](fig/dynd_struct_take_1_complicated_select_xy.svg)</center>

Something went wrong here, just like when we tried to put the stride in the fixed dimension type. The values don't match a C/C++ struct anymore, the second offset can't be 8. Similarly if the offset is 16, the type is different from the type the function we have takes.

# Field Offsets Are Like Array Strides

Just like when we first looked at where to put the stride for the fixed dimension, we have made a choice of where to put the offsets for the struct dtype. In this case, it was quite intuitive to simply do what C/C++ does, and put those offsets in the type. This is in fact what NumPy does.

This led to trouble, however, which was the exact same trouble that made us put the stride into the arrmeta. Therefore, we put these offsets in the arrmeta.

# DyND Struct With Offsets In The Arrmeta

Here's how it looks after we move the struct’s field offsets into the arrmeta. Our data A of type “{X: float64, DX: float64, Y: float64, DY: float64}” looks like:

<center>![DyND Struct Take 2](fig/dynd_struct_take_2.svg)</center>

# Struct Field Selection

Now if we do a field selection, extracting the X and Y fields, we’ll get a DyND struct whose type is “{X: float64, Y: float64}”, exactly as we wanted. The layout is not the same as a C/C++ struct, but views of the data are closed under field selection, just as views of strided arrays are closed under slicing. Here’s the result of that field selection:

<center>![Struct field selection](fig/dynd_struct_take_2_field_select_xy.svg)</center>

# Nesting Arrmeta

So far we’ve seen two forms of arrmeta, for fixed dimensions and for structs. A natural question arises, what happens when we have arrays of structs or structs of arrays? 